In [ ]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y
!pip install seaborn

In [ ]:
#Папки, куда будут создаваться новые фото
! mkdir ./aug0-1-2
! mkdir ./aug0-1-2/0
! mkdir ./aug0-1-2/1
! mkdir ./aug0-1-2/1/1
! mkdir ./aug0-1-2/2

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from PIL import Image, ImageOps
from random import randrange
from tqdm import tqdm
import os


In [ ]:
def random_crop(folder,folder_rez):
    files=os.listdir(folder)
    for image_name in files:
        
        img1 = Image.open(folder + '/' + image_name)
        img_size = img1.size
        x_max = img_size[0] - 600
        y_max = img_size[1] - 600
        if (x_max>10)&(y_max>10):
            for i in range(15):
                random_x = int(randrange(0, x_max))
                random_y = int(randrange(0, y_max))

                area = (random_x, random_y, random_x + 600, random_y + 600)
                c_img = img1.crop(area)

                fit_img_h = ImageOps.fit(c_img, (600, 600), Image.ANTIALIAS)
                fit_img_h.save('{}{}_{}_{}'.format(folder_rez,  random_x,random_y, image_name))





In [ ]:
#если нет бака, то его нет на всей фото
random_crop('./my_train0-1-2/0/','./aug0-1-2/0/')

In [ ]:
#если хреновое фото, то оно всё
random_crop('./my_train0-1-2/2/','./aug0-1-2/2/')

In [ ]:
#а тут повозиться придёться поэтому делаю больше
random_crop('./my_train0-1-2/1/','./aug0-1-2/1/1/')
random_crop('./my_train0-1-2/1/','./aug0-1-2/1/1/')


In [ ]:
#грузим промежуточную модель
model = tf.keras.models.load_model('./'+'pre_EfficientnetB7.hdf5')

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc , accuracy_score

In [ ]:
img_generator = keras.preprocessing.image_dataset_from_directory(
    './aug0-1-2/1/', 
    batch_size=8, 
    image_size=(600,600),
    shuffle=False,
)

file_paths = img_generator.file_paths

In [ ]:
file_paths

In [ ]:
predicts = []
pred_numpy = model.predict(img_generator)

for class_obj in pred_numpy:
    index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])
    predicts.append(index)

In [ ]:
#испольлзуя предварительную модель, удаляем лишние файлы

In [ ]:
prediction_df=pd.DataFrame(file_paths,columns=['file'])
prediction_df['class_pred']=predicts


In [ ]:
prediction_df[prediction_df['class_pred']==1]

In [ ]:
for i in prediction_df[prediction_df['class_pred']==1].index:
    shutil.copy(prediction_df['file'].loc[i], './aug0-1-2/1/')


In [ ]:
# всё что в папках ./aug0-1-2/0/ ./aug0-1-2/1/ ./aug0-1-2/2/, переносим в папку обучениыя вручную без подпапок ))))